In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Apparent Temperature Prediction
   The data contains weather parameters in the form of either continuous numerical values or descrete     classs. We are going to predict Apparent Temperature on a given day based on the other weather         parameters by employing Linear Regression, Decision Tree Regressor and Random Forests Regressor

In [ ]:
#Importing the important packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
#Reading the data (csv file) as Pandas DataFrame & showing its head 
data=pd.read_csv('/kaggle/input/szeged-weather/weatherHistory.csv')
data.head()

In [ ]:
#Getting some info about the data
data.info()

# Preprocessing the data

In [ ]:
# Defining a cutom function to apply some functions so that the data becomes ready to use in the models.
def processing_inputs(df):
    df=df.copy()
    
    #Droping these three columns bcz they can not be used for Linear regression model
    df=df.drop(['Summary','Daily Summary', 'Loud Cover'], axis=1)
    
    
    #Precip Type column has some Nan values so filling them with most repeated values and then converting 
    # to binary values
    df['Precip Type']=  df['Precip Type'].fillna(df['Precip Type'].mode()[0])
    df['Precip Type']=df['Precip Type'].apply(lambda x: 1 if x=='snow' else 0)
    
    # Converting the Formatted Date into datetime format and getting seprate columns for each parameter
    df['Formatted Date']= pd.to_datetime( df['Formatted Date'], format= '%Y-%m-%d %H:%M:%S.%f %z')
    
    df['Year'] = df['Formatted Date'].apply(lambda x: x.year)
    df['Month'] = df['Formatted Date'].apply(lambda x: x.month)
    df['Day'] = df['Formatted Date'].apply(lambda x: x.day)
    df['Hour'] = df['Formatted Date'].apply(lambda x: x.hour)
    df= df.drop('Formatted Date', axis=1)
    
    #Seprating columns to features and output
    X=df.drop('Apparent Temperature (C)', axis=1).copy()
    y=df['Apparent Temperature (C)'].copy()
    
    #Spliting the data as training and testing datasets
    X_train, X_test, y_train, y_test= train_test_split(X,y, train_size=0.7, random_state=123)
    
    #Applying StandardScaler to scale the features so that mean becomes zero and stand.devition as 1.
    lm=StandardScaler()
    X_train=pd.DataFrame(lm.fit_transform(X_train), columns=X_train.columns)
    X_test= pd.DataFrame(lm.transform(X_test), columns=X_test.columns)
    
    return  X_train, X_test, y_train, y_test

In [ ]:
#Passing the data to our custom preprocessing function so that all the operation done above take place.
X_train, X_test, y_train, y_test=processing_inputs(data)

In [ ]:
# Have a look at X_train now
X_train

In [ ]:
# creating a dics to fit the data for all the three models at a time using for loop
models={"Linear Regression": LinearRegression(), "DecisionTreeRegressor" :DecisionTreeRegressor(), "KNeighborsregressor": KNeighborsRegressor()}
for p, q in models.items():
    q.fit(X_train, y_train)
    print( p +  " trained and predicted values are given below:")
    pred=q.predict(X_test)
    model_index=list(models.keys()).index(p)
    plt.figure(figsize=(10,5))
    plt.subplot(3,1, model_index+1)
    plt.scatter(pred, y_test)
    print(pred)

In [ ]:
#Using the above dics and new for loop calculating R2 score for all the three models
for p, q in models.items():
    print(p, " R2 : {:4f}".format(q.score(X_test, y_test)))

In [ ]:
#decision Tree regressor did the best.